In [1]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings,ChatGoogleGenerativeAI
import os
from dotenv import load_dotenv
from langchain.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate

load_dotenv()

GOOGLE_API_KEY=os.environ["GOOGLE_API_KEY"]
persist_directory = 'docs/chroma/'

In [19]:
def gen_context(vectordb,question):
    docs = vectordb.similarity_search(question,k=3)
    doc_list= []
    context=""
    for i in range(len(docs)):
        #print(docs[i].page_content)
        context =context + docs[i].page_content +"\n"
    for doc in docs:
        #print(doc.metadata)
        doc_list.append(doc.metadata)
    return context,doc_list

In [2]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

vectordb = Chroma(persist_directory=persist_directory, embedding_function=embeddings)

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    max_tokens=250,
    temperature=0.5,
)

In [39]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant that answers questions provided by the user. You must use the available {context} for providing answers. Do not provide any other answers if they are not present in {context}.",
        ),
        ("human", "context:{context}\nquestion:{question}"),
    ]
)

chain = prompt | llm 

In [40]:
question= "Hi can you tell some solar panel issues i can encounter?"
context,doc_list=gen_context(vectordb,question)
ai_msg=chain.invoke({
    "question": question,
    "context": context
    })

In [41]:
print(ai_msg.content)

Here are some solar panel issues you might encounter:

* **Dust on Solar Panel:**  Dust can accumulate on the panels, reducing their efficiency. Regular cleaning is important.
* **Connection issue with main board:**  The wiring connecting the panels to the main board can become loose or damaged.
* **Solar Panel Damage:**  Physical damage to the panels, such as cracks or broken glass, can significantly impact their performance. 



In [42]:
print(context)
for i in range(len(doc_list)):
    print(doc_list[i])

ii.
Current
sensor
not
working
iii.
Motor
is
not
working
d.
Malfunction
of
Valve
device
i.
Valve
is
damaged
ii.
Valve
is
not
fully
open
4.
SolarTrack
a.
Internet
is
not
working
i.
Connection
is
disconnected
ii.
Firewall
issues
b.
Inverter
Modbus
not
working
i.
Inverter
is
off
ii.
Modbus
is
damaged
iii.
Switch
is
damaged
iv.
Inverter
IP
configurations
changed
c.
Main
board
issue
i.
Memory
overflow
ii.
Stuck
in
program
iii.
Heat
issue
in
the
Main
board
d.
Device
is
restarted
Solutions
for
those
Failure
Scenarios
Device
Failure
scenarios
1.
Common
:
a.
GSM
connection
failed
i.
SIM
damaged
ii.
GSM
module
damaged
iii.
GSM
module
loss
connection
with
main
board
iv.
Connection
disconnected
due
to
payment
issue
v.
GSM
capacitor
damaged
b.
Data
drops
i.
Coverage
issue
c.
Battery
charging
reducing
d.
Battery
dead
i.
Direct
Power
source
is
off
ii.
Battery
damaged
e.
Internal
temperature
is
increasing
f.
Main
board
failed
i.
Internal
temperature
is
high
ii.
Water
inside
g.
Server
down
i.
MQTT
Brok

In [6]:
from qa_system import initialise_qa

In [9]:
question= "Can you give me all the records regarding solar panels?"
result = initialise_qa(question)
print(result)

I have one record regarding solar panels.

- **Record ID:** 1
- **Device ID:** 101
- **Device Name:** Solar Panel
- **Description:** There are dusts in the solar panel.
- **Notifier Date Time:** 2024-07-31 11:37:19 



In [28]:
# Define the prompt template
prompt_template = """
You are a database assistant. A user has a SQLite database with a table called 'Records'.
The table has the following columns: records_id, device_id, device_name, description, notifier_date_time.
If the user query request anything that does not include the database, then provide your as None.
Do not generate any other than the SQL query required.


Please generate an SQL query to retrieve records based on the following user request:
{user_request}
"""

prompt = PromptTemplate(template=prompt_template, input_variables=["user_request"])

# Create the LangChain
chain = prompt| llm

In [10]:
def clean_sql_query(query):
    # Remove triple backticks and surrounding whitespace
    return query.replace("```sql", "").replace("```", "").strip()

In [42]:
def query_database(user_request):
    # Generate the SQL query
    response = chain.invoke({"user_request": user_request})
    generated_query = response["text"].strip()
    print(f"Generated SQL Query: {generated_query}")

    # Connect to the database
    conn = sqlite3.connect(database_path)
    df = pd.read_sql_query(generated_query, conn)
    conn.close()

    return df

# User request
user_request = "Retrieve all records for device_id 101"
results = query_database(user_request)

TypeError: 'AIMessage' object is not subscriptable

In [29]:
user_request = "What is the solution for bug infestation"
response = chain.invoke({"user_request": user_request})
generated_query = response.content.strip()
print(f"Generated SQL Query: {generated_query}")
cleaned_query = clean_sql_query(generated_query)
print(f"Cleaned SQL Query: {cleaned_query}")

Generated SQL Query: ```sql
SELECT * FROM Records WHERE description LIKE '%bug infestation%'
```
Cleaned SQL Query: SELECT * FROM Records WHERE description LIKE '%bug infestation%'


In [30]:
conn = sqlite3.connect(database_path)
df = pd.read_sql_query(cleaned_query, conn)
df

,records_id,device_id,device_name,description,notifier_date_time


In [31]:
# Function to convert DataFrame to text
def dataframe_to_text(df):
    rows = []
    for _, row in df.iterrows():
        rows.append(f"Record ID: {row['records_id']}, Device ID: {row['device_id']}, Device Name: {row['device_name']}, Description: {row['description']}, Notifier Date Time: {row['notifier_date_time']}")
    return "\n".join(rows)


# Define the prompt template for generating descriptions
desc_prompt_template = """
You are an assistant that summarizes database records. If the records contain None, then you shall provide the answer as None. Please provide a bulleted description for each record given below:

{records}
"""

desc_prompt = PromptTemplate(template=desc_prompt_template, input_variables=["records"])
# Create the LangChain for generating descriptions
desc_chain = desc_prompt|llm

In [32]:
records_text = dataframe_to_text(df)
records_text

''

In [33]:
response = desc_chain.invoke({"records": records_text})
print(response.content)

Please provide the database records you would like me to summarize. I'm ready to help! 😊 



In [1]:
from utils.records import records_output

In [2]:
question="Hi can you give me recent records?"
result = records_output(question)

In [3]:
print(result)

- **Record ID:** 1
- **Device ID:** 3
- **Device Name:** Device_B
- **Description:** This is a test description
- **Notifier Date Time:** 2024-07-31 11:05:05 

